In [ ]:
#!pip install yfinance

In [2]:
from flask import Flask, request, redirect, render_template_string,jsonify,url_for
import io
import seaborn as sns
import base64
from io import StringIO
import yfinance as yf
import pandas as pd
import humanize
import matplotlib.pyplot as plt
import plotly.graph_objs as go


app = Flask(__name__)

# Captures a string name from the URL.
@app.route('/getCompanyInfo/<compnysym>', methods=['GET'])
def get_companyinfo(compnysym):
    # create ticker for Apple Stock
    try:
        GetCompanyInformation = yf.Ticker(compnysym)
    # get data of the most recent date
        companyInfo_dict = dict(GetCompanyInformation.info.items())
        print(companyInfo_dict)
        filter_key = ['shortName', 'longBusinessSummary','industry','sector']
        company_details = {key: companyInfo_dict[key] for key in filter_key if key in companyInfo_dict}
        df_companydetails = pd.DataFrame.from_dict(company_details, orient='index')
        companyOfficers_details_list = companyInfo_dict.get('companyOfficers')
        print(companyOfficers_details_list)
        
        #Extract name and title from list of dictionaries companyOfficers
        dict_key = 'name'
        companyOfficers_details_names = [d.get(dict_key) for d in companyOfficers_details_list if dict_key in d]
        series_companyOfficers_details_names = pd.Series(companyOfficers_details_names)
    
        dict_key = 'title'
        companyOfficers_details_titles = [d.get(dict_key) for d in companyOfficers_details_list if dict_key in d]
        series_companyOfficers_details_titles = pd.Series(companyOfficers_details_titles)
        
        frame = {'Name': series_companyOfficers_details_names,
                 'Title': companyOfficers_details_titles}
        df_companydetails_html = df_companydetails.to_html()
        df_companyofficers = pd.DataFrame(frame)
        df_companyofficers_html = df_companyofficers.to_html()
        print(df_companyofficers)
        print(df_companydetails_html)
        html_str = df_companydetails_html+'<div></div>'+df_companyofficers_html
    except:
        html_str= '<h1>An Error occured while calling the endpoint, Please check if symbol is correct </h1>'
    return render_template_string(html_str)

@app.route('/getMarketInfo/<compnysym>', methods=['GET'])
def get_marketState(compnysym):
    # create ticker for Apple Stock
    try:
        GetCompanyInformation = yf.Ticker(compnysym)
    # get data of the most recent date
        companyInfo_dict = dict(GetCompanyInformation.info.items())
        filter_key = ['marketCap', 'regularMarketVolume','fiftyTwoWeekLow','fiftyTwoWeekHigh','enterpriseValue','bookValue','ebitda','quickRatio','debtToEquity','regularMarketPrice','epsCurrentYear','regularMarketChange','regularMarketChangePercent']
        market_details = {key: companyInfo_dict[key] for key in filter_key if key in companyInfo_dict}
        df_marketdetails = pd.DataFrame.from_dict(market_details, orient='index')
        df_marketdetails[0][0] = humanize.intword(df_marketdetails[0][0])
        df_marketdetails[0][1] = humanize.intword(df_marketdetails[0][1])
        df_marketdetails[0][4] = humanize.intword(df_marketdetails[0][4])
        df_marketdetails[0][6] = humanize.intword(df_marketdetails[0][6])
        print(df_marketdetails)
        df_marketdetails_html = df_marketdetails.to_html()
        html_str = df_marketdetails_html
    except:
        html_str = '<h1>An Error occured while calling the endpoint, Please check if symbol is correct </h1>'
    return render_template_string(html_str)


@app.route('/submitDates', methods=['GET','POST'])
def submit():
    if request.method == 'POST':
        data = request.get_json()
        print(data)
        compnysym = data.get('compnysym', 'No symbol')
        startDate = data.get('startDate', 'No start')
        endDate = data.get('endDate', 'No end')
        hist_data = yf.download(compnysym, start=startDate, end=endDate)
        print(hist_data)
        hist_data.to_csv(f"{compnysym}_historical_data.csv")
        return jsonify({'response': f'You submitted: {startDate},{endDate},{compnysym}'})
    else:
        return '''
             <script type="text/javascript">
                function setJsonContentHeader(event) {
                event.preventDefault(); // Prevent the default form submission
            
                const form = event.target;
                const formData = new FormData(form);
                const jsonData = {};
            
                formData.forEach((value, key) => {
                    jsonData[key] = value;
                });
            
                fetch(form.action, {
                    method: form.method,
                    headers: {
                        'Content-Type': 'application/json'
                    },
                    body: JSON.stringify(jsonData)
                })
                .then(response => response.json())
                .then(data => {
                    console.log('Success:', data);
                })
                .catch((error) => {
                    console.error('Error:', error);
                });
            }
            </script>
            <form method="post" action="/submitDates"  onsubmit="setJsonContentHeader(event)">
              <label for="compnysym">Company symbol:</label>
              <input type="text" id="compnysym" name="compnysym">
              <label for="startDate">Start Date:</label>
              <input type="date" id="startDate" name="startDate">
              <label for="endDate">End Date:</label>
              <input type="date" id="endDate" name="endDate">
              <input type="submit"/>
            </form>        '''
            
@app.route('/performanalysis/<compnysym>', methods=['GET'])
def performsstockanalysis(compnysym):
    stock = yf.Ticker(compnysym)
    hist = stock.history(period='2y')
    plt.figure(figsize=(10, 5))
    plt.plot(hist.index, hist['Close'], label='Close Price')
    plt.title(f'{compnysym} Stock Price Trend Over Last 2yrs')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.legend()
    plt.grid(True)
    plt.savefig('static/stock_trend.png')
    plt.close()

     # Calculate the Simple Moving Averages
    hist['MA10'] = hist['Close'].rolling(window=10).mean()
    hist['MA20'] = hist['Close'].rolling(window=20).mean()

     # Create the plot
    fig = go.Figure()

    # Add traces
    fig.add_trace(go.Scatter(x=hist.index, y=hist['Close'], mode='lines', name='Close'))
    fig.add_trace(go.Scatter(x=hist.index, y=hist['MA10'], mode='lines', name='MA10'))
    fig.add_trace(go.Scatter(x=hist.index, y=hist['MA20'], mode='lines', name='MA20'))

    # Update layout
    fig.update_layout(title=f'{compnysym} Stock Price and Moving Averages',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      template='plotly_dark')

    graph_html = fig.to_html(full_html=False)
    print(graph_html)
    
    hist['Daily Return'] = hist['Close'].pct_change()
    plt.figure(figsize=(10, 6))
    plt.plot(hist['Daily Return'])
    plt.title('Stock Daily Returns')
    plt.xlabel('Date')
    plt.ylabel('Daily Return')
    plt.grid(True)
    plt.savefig('static/daily_return.png')
    plt.close()

    
    html = '''
    <html>
    <head><title>Stock Price Trend</title></head>
    <body>
        <h1>{{ compnysym }} Stock Price Trend </h1>
        <img src="{{ url_for('static', filename='stock_trend.png') }}" alt="Stock Price Trend">
        <h1>{{ ticker_symbol }} Stock Price and Moving Averages</h1>
        {{ graph_html | safe }}
        <h1>{{ compnysym }} Volatility </h1>
        <img src="{{ url_for('static', filename='daily_return.png') }}" alt="Daily Return Trend">
    </body>
    </html>
    '''
    return render_template_string(html, compnysym=compnysym,graph_html=graph_html)
if __name__ == '__main__':
    app.run(debug=False)

    
# Testing with curl
# Replace <name> with any username you like.
# curl http://localhost:5000/getCompanyInfo/AAPL

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

127.0.0.1 - - [13/Mar/2025 14:11:50] "GET /performanalysis/NVDA HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 14:11:50] "GET /static/stock_trend.png HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2025 14:11:51] "GET /static/daily_return.png HTTP/1.1" 200 -
